In [2]:
# llama-parse is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os
import tqdm
import pandas as pd

from openai import OpenAI
clientDS = OpenAI(api_key=os.environ['DEEPSEEK_API_KEY'],base_url="https://api.deepseek.com")
clientNV = OpenAI(api_key=os.environ['NVIDIA_API_KEY'],base_url="https://integrate.api.nvidia.com/v1")

In [4]:
docFolderDir = './S03_QuestionGeneration/'
lsDocDir = [f for f in os.listdir(docFolderDir)]
lsDocDir

['Industrial_Gas_Turbines_AMY_Razak',
 'msd_servo_drive',
 'pcs7_compendium_part_a_en-US_en-US',
 'pcs7_compendium_part_b_en-US_en-US']

In [5]:
for i in range(len(lsDocDir)):
    # i=0
    # CHOOSE FOLDER
    docfolder = lsDocDir[i]
    # CREATE FOLDER IF NOT EXISTS
    if not os.path.exists(f"./S04_AnswerScoring/{docfolder}/"):
        os.makedirs(f"./S04_AnswerScoring/{docfolder}/")
    # LIST ALL CSV IN FOLDER
    lscsv = [f for f in os.listdir(f"./S03_QuestionGeneration/{docfolder}") if f.endswith('.csv')]
    # LOOP EACH CSV
    for j in range(len(lscsv)):
        # PRINT
        print(docfolder,j,len(lscsv))
        # PARSE
        csvj = lscsv[j]
        # ONLY WORK IF NOT ALREADY EXISTS
        if not os.path.isfile(f"./S04_AnswerScoring/{docfolder}/{csvj}"):    
            # LOAD CSV
            dfraw = pd.read_csv(f"./S03_QuestionGeneration/{docfolder}/{csvj}")
            if 'Unnamed: 0' in dfraw.columns:
                dfraw = dfraw.drop('Unnamed: 0', axis=1)
            # MELT
            dfmelt = dfraw.melt(
                id_vars=["NODE_FILE_NAME", "NODE_NUMBER", "NODE_HASH", "NODE_TEXT", "Q"],
                value_vars=["A1", "A2", "A3", "A4", "A5"],
                var_name="A_NUMBER",
                value_name="A")
            dfmelt = dfmelt.sort_values(['Q','A_NUMBER']).reset_index(drop=True)
            # PREP COLUMN
            dfmelt['helpfulness'] = None
            dfmelt['correctness'] = None
            dfmelt['coherence'] = None
            dfmelt['complexity'] = None
            dfmelt['verbosity'] = None
            # LOOP GET SCORE
            for k in tqdm.tqdm(range(len(dfmelt))):
                # PARSE Q/A
                question = dfmelt['Q'].iat[k]
                answer = dfmelt['A'].iat[k]
                # BUILD MESSAGE
                messages = [{"role": "user", "content": question},
                            {"role": "assistant", "content": answer}]
                # GET RESPONSE
                response = clientNV.chat.completions.create(model="nvidia/nemotron-4-340b-reward", messages=messages)
                logprobs = response.choices[0].logprobs.content
                score_dict = {}
                for score in logprobs:
                    score_dict[score.token] = score.logprob
                # SAVE RESPONSE
                dfmelt['helpfulness'].iat[k] = score_dict['helpfulness']
                dfmelt['correctness'].iat[k] = score_dict['correctness']
                dfmelt['coherence'].iat[k] = score_dict['coherence']
                dfmelt['complexity'].iat[k] = score_dict['complexity']
                dfmelt['verbosity'].iat[k] = score_dict['verbosity']
            # SAVE CSV
            dfmelt.to_csv(f"./S04_AnswerScoring/{docfolder}/{csvj}")
        

Industrial_Gas_Turbines_AMY_Razak 0 381
Industrial_Gas_Turbines_AMY_Razak 1 381
Industrial_Gas_Turbines_AMY_Razak 2 381


100%|██████████| 50/50 [00:25<00:00,  1.99it/s]


Industrial_Gas_Turbines_AMY_Razak 3 381


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


Industrial_Gas_Turbines_AMY_Razak 4 381


...

100%|██████████| 50/50 [00:25<00:00,  1.98it/s]


pcs7_compendium_part_b_en-US_en-US 79 81


100%|██████████| 50/50 [00:23<00:00,  2.11it/s]


pcs7_compendium_part_b_en-US_en-US 80 81


100%|██████████| 50/50 [00:23<00:00,  2.11it/s]
